In [1]:
# !pip install torch
# !pip install tqdm
# !pip install transformers
# !pip install scikit-learn
# !pip install pandas
# !pip install wandb
# !pip install evaluate
# !pip install transformers[torch]

In [2]:
import pandas as pd
import pickle
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoConfig
import evaluate
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report
from customModel import customBertForSequenceClassification, customRobertaForSequenceClassification,customGPT2ForSequenceClassification, customElectraForSequenceClassification
from CustomTraniner import CustomTrainer
from transformers.configuration_utils import PretrainedConfig
import wandb
import random
from transformers.models.auto.modeling_auto import MODEL_FOR_SEQUENCE_CLASSIFICATION_MAPPING
from frontModelCustom import frontModelDataset, data_labels



/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/leesk/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
def set_seed(seed:int = 42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
set_seed(42)

### Data Process

In [5]:
data = pd.read_csv('./genre_11_tempo_4.csv')
data.head()

,singer,song_name,index,emotion,caption,tempo,instruments,tempo(int),tempo(category),genre,sub_genre
0,"10,000 Maniacs",A Campfire Song,2,nostalgia,"The melody is mellow and soothing, with a gent...",Tempo_135.0,"['Program_-1', 'Program_16', 'Program_24', 'Pr...",135,Allegro,Rock,Alternative rock
1,101 Strings,Theme From The Godfather,3,nostalgia,Elegant and sweeping orchestral melody with a ...,Tempo_80.0,"['Program_-1', 'Program_0', 'Program_12', 'Pro...",80,Moderato,Pop,Pop
2,10cc,Dreadlock Holiday,4,excitement,"catchy and upbeat, featuring a memorable guita...",Tempo_103.0,"['Program_-1', 'Program_0']",103,Moderato,Rock,Reggae rock
3,10cc,I'm Not In Love,5,nostalgia,A catchy and memorable tune with a simple yet ...,Tempo_65.0,"['Program_-1', 'Program_0', 'Program_122', 'Pr...",65,Andante,Rock,Soft rock
4,10cc,The Things We Do for Love,6,love,"The melody is catchy and memorable, with a ble...",Tempo_111.0,"['Program_-1', 'Program_0', 'Program_16', 'Pro...",111,Moderato,Rock,Soft rock


In [6]:
with open('labels.pkl','wb') as f:
    pickle.dump(data.emotion.unique(),f)
    pickle.dump(data['tempo(category)'].unique(),f)
    pickle.dump(data['genre'].unique(),f)

In [7]:
# with open('labels.pkl','rb') as f:
#     pickle.dump(data.emotion.unique(),f)
#     pickle.dump(data['tempo(category)'].unique(),f)
#     pickle.dump(data['genre'].unique(),f)

In [8]:
# id2label_emotion = {k:l for k, l in enumerate(data.emotion.unique())}
# label2id_emotion = {l:k for k, l in enumerate(data.emotion.unique())}
# id2label_tempo = {k:l for k, l in enumerate(data['tempo(category)'].unique())}
# label2id_tempo = {l:k for k, l in enumerate(data['tempo(category)'].unique())}
# id2label_genre = {k:l for k, l in enumerate(data['genre'].unique())}
# label2id_genre = {l:k for k, l in enumerate(data['genre'].unique())}

In [9]:
# train_data, valid_data = train_test_split(data, stratify=data['emotion'],test_size= 0.1, random_state=42)

In [10]:
class customDataset_after():
    def __init__(self, data, tokenizer):
        self.tokenizer = tokenizer
        self.dataset = []
        datas = []
        self.labels1 = []
        self.labels2 = []
        self.labels3 = []
        for idx, df in tqdm(data.iterrows()):
            label1 = [0. for _ in range(len(id2label_emotion))]
            label2 = [0. for _ in range(len(id2label_tempo))]
            label3 = [0. for _ in range(len(id2label_genre))]
            datas.append(df.caption)
            label1[label2id_emotion[df.emotion]] = 1.
            label2[label2id_tempo[df['tempo(category)']]] = 1.
            label3[label2id_genre[df['genre']]] = 1.
            self.labels1.append(label1)
            self.labels2.append(label2)
            self.labels3.append(label3)
        self.dataset =  tokenizer(datas,padding=True, truncation=True,max_length=512 ,return_tensors="pt").to('cuda')
        self.labels1= torch.tensor(self.labels1)
        self.labels2= torch.tensor(self.labels2)
        self.labels3= torch.tensor(self.labels3)
    def __len__(self):
        return len(self.labels1)
    
    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.dataset.items()}
        item['labels1'] = self.labels1[idx].clone().detach()
        item['labels2'] = self.labels2[idx].clone().detach()
        item['labels3'] = self.labels3[idx].clone().detach()
        return item

In [11]:
emotion , tempo, genre = data_labels('labels.pkl')

In [12]:
BASE_MODEL = 'FacebookAI/roberta-large'

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
config = AutoConfig.from_pretrained(BASE_MODEL)
config.num_labels1 = len(emotion)
config.num_labels2 = len(tempo)
config.num_labels3 = len(genre)
# model = customBertForSequenceClassification.from_pretrained(BASE_MODEL, config= config).to(device)
model = customRobertaForSequenceClassification.from_pretrained(BASE_MODEL, config= config).to(device)
# model = customElectraForSequenceClassification.from_pretrained(BASE_MODEL, config= config).to(device)
# model = customGPT2ForSequenceClassification.from_pretrained(BASE_MODEL, config= config).to(device)
# tokenizer.pad_token = tokenizer.eos_token
# model.config.pad_token_id = model.config.eos_token_id

Some weights of customRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier1.bias', 'classifier1.weight', 'classifier2.bias', 'classifier2.weight', 'classifier3.bias', 'classifier3.weight', 'dense.bias', 'dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
type(config)

transformers.models.roberta.configuration_roberta.RobertaConfig

In [14]:
data2 = data.copy()
emotion_data = data2.groupby('emotion').sample(frac=0.05, random_state=42)
tempo_data = data2.groupby('tempo(category)').sample(frac=0.05, random_state=42)
genre_data = data2.groupby('genre').sample(frac=0.05, random_state=42)
index_total = set(emotion_data.index) | set(tempo_data.index) | set(genre_data.index)
valid_data = data2.iloc[list(index_total)]
train_data = data2.drop(list(index_total)).sample(frac=1, random_state=42)

In [15]:
# dataset_train = frontModelDataset(train_data, tokenizer =tokenizer)
# dataset_valid = frontModelDataset(valid_data, tokenizer =tokenizer)

dataset_train = frontModelDataset(train_data, tokenizer =tokenizer)
dataset_valid = frontModelDataset(valid_data, tokenizer =tokenizer)



8552it [00:00, 13272.91it/s]
1419it [00:00, 13265.57it/s]


In [16]:
GLOBAL_SCORE_INDICES = range(0,17)
CAUSE_INDICES = range(17, 25)
def get_preds_from_logits(logits):
    ret = np.zeros(logits.shape)
    
    # The first 5 columns (GLOBAL_SCORE_INDICES) are for global scores. They should be handled with a multiclass approach
    # i.e. we fill 1 to the class with highest probability, and 0 into the other columns
    best_class = np.argmax(logits, axis=1)
    ret[list(range(len(ret))), best_class] = 1
    # The other columns are for causes and emotions. They should be handled with multilabel approach.
    # i.e. we fill 1 to every class whose score is higher than some threshold
    # In this example, we choose that threshold = 0
    
    return ret

In [17]:
# clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

# def sigmoid(x):
#    return 1/(1 + np.exp(-x))

# def compute_metrics(eval_pred):

#    predictions, labels = eval_pred
#    predictions = sigmoid(predictions)
#    predictions = (predictions > 0.5).astype(int).reshape(-1)
#    return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))

In [18]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    final_metrics = {}
    
    # Deduce predictions from logits
    predictions_emotion = get_preds_from_logits(logits[0])
    predictions_tempo = get_preds_from_logits(logits[1])
    predictions_genre = get_preds_from_logits(logits[2])
    
    # Get f1 metrics for global scoring. Notice that f1_micro = accuracy
    final_metrics["f1_emotion"] = f1_score(labels[0], predictions_emotion, average="micro")
    
    # Get f1 metrics for causes
    final_metrics["f1_tempo"] = f1_score(labels[1], predictions_tempo, average="micro")
    

    # The global f1_metrics
    final_metrics["f1_genre"] = f1_score(labels[2], predictions_genre, average="micro")

    final_metrics['fi_total'] = (final_metrics["f1_emotion"] + final_metrics["f1_tempo"] + final_metrics["f1_genre"])/3
    
    # Classification report
    # print("Classification report for global scores: ")
    # print(classification_report(labels[:, GLOBAL_SCORE_INDICES], predictions[:, GLOBAL_SCORE_INDICES], zero_division=0))
    # print("Classification report for causes: ")
    # print(classification_report(labels[:, CAUSE_INDICES], predictions[:, CAUSE_INDICES], zero_division=0))
    return final_metrics

In [19]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [20]:
print(config.num_labels1, config.num_labels2, config.num_labels3)

wandb.init(project="Final project", entity="sanggang",name = "warmup_100_"+BASE_MODEL)

training_args = TrainingArguments(

   output_dir="my_awesome_model",
   save_steps=300,
   eval_steps = 300, 
   warmup_steps=500,
   logging_steps=100,
   learning_rate=5e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=10,
   weight_decay=0.01,
   evaluation_strategy='steps',
   load_best_model_at_end = True,
   save_total_limit = 2,
   report_to="wandb",
   metric_for_best_model='fi_total',
   # run_name=BASE_MODEL, 
)

trainer = CustomTrainer(

   model=model,
   args=training_args,
   train_dataset=dataset_train,
   eval_dataset=dataset_valid,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


# trainer = Trainer(

#    model=model,
#    args=training_args,
#    train_dataset=dataset_train,
#    eval_dataset=dataset_valid,
#    tokenizer=tokenizer,
#    data_collator=data_collator,
#    compute_metrics=compute_metrics,
# )



trainer.train()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


10 4 11


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: leesk9663 (sanggang). Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss,Validation Loss,F1 Emotion,F1 Tempo,F1 Genre,Fi Total
300,0.988200,0.209489,0.579281,0.379845,0.393939,0.451022
600,0.987500,0.205776,0.546159,0.361522,0.432699,0.446794
900,0.949600,0.192495,0.593376,0.410853,0.452431,0.485553
1200,0.925200,0.191204,0.588443,0.412967,0.474278,0.491896
1500,0.941300,0.190577,0.579281,0.374207,0.477097,0.476862
1800,0.906900,0.189389,0.599013,0.415786,0.484144,0.499648
2100,0.906400,0.186391,0.603242,0.369274,0.501762,0.491426
2400,0.892300,0.194968,0.584214,0.401691,0.432699,0.472868
2700,0.873600,0.181848,0.606765,0.399577,0.508104,0.504816
3000,0.848800,0.182850,0.599718,0.400987,0.520789,0.507165


TrainOutput(global_step=3210, training_loss=0.9363039815908651, metrics={'train_runtime': 1990.8604, 'train_samples_per_second': 25.774, 'train_steps_per_second': 1.612, 'total_flos': 5977872281659392.0, 'train_loss': 0.9363039815908651, 'epoch': 6.0})